In [ ]:
# Understanding Off topic
import numpy as np

a = np.arange(6).reshape(2, 3)
print(a)
print(a[:, np.newaxis, 2])

In [ ]:
# function to fetch the data
import os
import tarfile
import urllib

DOWNLOAD_ROOT = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/'
HOUSING_PATH = os.path.join("datasets", "housing") # Creates a path in the directory, consiting of a datasets folder, containing a housing folder
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [ ]:
# helper function for fetching the data
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [ ]:
# Loads the data into Pandas
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
# Fetches and loads the data
fetch_housing_data()
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe()

In [ ]:
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
# income categories 1.5-6.0, this represents 15,000-60,000
# Creates an income category attribute with 5 categories (labeled from 1 to 5): 
# category 1 ranges from 0 to 1.5 (i.e less than 15,000)
# category 2 ranges from 1.5 to 3, and so on.
import numpy as np
housing['income_cat'] = pd.cut(housing['median_income'], bins=[0, 1.5, 3.0, 4.5, 6, np.inf], labels=[1, 2, 3, 4, 5])
housing['income_cat'].hist()


In [ ]:
# stratified sampling based on the income category
# stratum - a level or class to which people are assigned according to their social status, education, or income.
# stratified - a common sampling technique used by researchers when trying to draw conclusions from different sub-groups or strata.

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index] # Here we get the trained set from the housing dataset
    strat_test_set = housing.loc[test_index]
    
strat_test_set['income_cat'].value_counts() / len(strat_test_set)

In [ ]:
# removes income_Cat attribute so the data is back to its original state:
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)

## What is the axis attribute?
I saw this a bunch of times, here is a reminder
<img src="img/axis_attr.png" alt="Drawing" style="width: 500px;"/>

In [ ]:
# housing.plot(kind='scatter', x='longitude', y='latitude) - without alpha, cannot see the density
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1)

In [ ]:
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4,
            s=housing["population"]/100, label='population', figsize=(10,7),
            c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()

In [ ]:
# corr() -computes the standard correlation coefficient (also called Pearson's r), between every pair of attributes
# how much each attribute correlates with the median house value:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

# About the Correlation coeficient
The correlation coefficient ranges from -1 to 1. 

When the value is close to 1, it means there is a positive correlation. For example, the above situation you can see the median house value tends to go up when the median income goes up. 

When the value is close to -1, it means there is a negative correlation. For example, you can see a small negative correlation in latitude (i.e., when the prices have a slight tendency to go down when you go North).

When the value is close to 0, it means there is no linear correlation. 

Again, this is only measures linear correlation. Not nonlinear.

Here is the correlation coeficient when plotted. 
<img src="img/correlation_plot.png" alt="Drawing" style="width: 500px;"/>

In [ ]:
# Another way to check for correlation between attributes is to use the pandas: scatter_matrix() func
# This plots every numerical attribute against every other numerical attribute.

from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))
# The diagnol graphs (top-right -> bottom-left):
# These are varaibles plotted against themselves. So instead of showing the correlation, Pandas dislpays a histogram
# of each attribute

In [ ]:
# Shows the plot just for median income. Showing a very strong (positive) correlation.
housing.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.1)

In [ ]:
# Create new useful attributes 
housing["rooms_per_household"] = housing['total_rooms']/housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms']/housing['total_rooms']
housing['population_per_household'] = housing['population']/housing['households']

In [ ]:
# view correlation
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)
# we can see houses with a lower bedroom/room ratio tend to be more expensive

In [ ]:
# Drops the median house value for the training set and assigns to housing variable
# Copys the median house value to housing_labels to easy access.
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()
# Here, we are predicting the median house values. So we drop it from the housing set, and add it to the labels

In [ ]:
"""
Drops the missing value datasets
3 Options:
    1. get rid of corresponding districts
    2. get rid of whole attribute
    3. set the values to some  value(zero, the mean, the median)
    
housing.dropna(subset=['total_bedrooms'])   # option 1
housing.drop('[total_bedrooms'], axis=1)    # option 2
median = housing['total_bedrooms'].median() # option 3
housing['total_bedrooms'].fillna(median, inplace=True) 

When using drop only, it drops the columns/rows you define
When using dropna, it removes all entries with NaN values (or null in general)
"""
# Scikit-LEarn provides a class to take care of missing values: SimpleImputer

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

# Since median can only be calculated with numerical data, we have to drop ocean_proximity and calculate the median
housing_numerical = housing.drop('ocean_proximity', axis=1)

In [ ]:
# imputer simply computers the median of each attribute and stored the results in its statistics_ variable.
imputer.fit(housing_numerical)

# Provides the same information
imputer.statistics_
housing_numerical.median().values

In [ ]:
# We can use this "trained" inputer to transform the training set by replacing missing values with the learned medians
X = imputer.transform(housing_numerical)

# The result above is a plain NumPy array containing the transformed features.
# We can put the NumPy array back into a pandas DataFrame by doing the following:

housing_tr = pd.DataFrame(X, columns=housing_numerical.columns, index=housing_numerical.index)

# Scikit-Learn Design

### Consistency:
>Estimators:
    >>Any object that can estimate some parameters based on a dataset is called an estimator( e.g., an imputer is an estimator). The estimation is performed by the fit() method, and it takes only a dataset as a parameter (or two for supervised learning algorithms; the second dataset contains the labels). Any other parameter needed to guide the estimation process is considered a hyperparameter (such as an imputer's strategy), and it must be aset as an instance varaible (generally via  a constructor parameter). 
    
>Transformers:
    >>Some estimators (such as an imputer) can also transform a dataset; these are called transformers. The transformation is performed by the transform() method with the dataset to transform as the parameter. All transformers also have a convenience method c alled fit_transform() that is equivalent to calling fit() and then transform() (but sometimes fit_transform() is optimized and runs much faster
    
>Predictors:
    >>Finally, some estimators, given a dataset, are capable of making predictions; they are called predictors. For example, the LinearRegression model in the previous chapter was a predictor. A predictor has a predict() method that takes a dataset of new instances and returns a dataset of corresponding predictions. It also has a score() method that measures the quality of the predictions, given a test set( and the corresponding labels, in the case of supervised learning algorithms).
    
### Inspection:
>All the estimator's hyperparameters are accessible directly via public instance vairiables (e.g., imputer.strategy), and all estimator's learned paramters are accessible via public instance variables with an underscore suffic (e.g., imputer.statistics_).

### Nonproliferation of classes
>Datasets are represented as NumPy arrays or SciPy sparse matricies, instead of homemade classes. Hyperparameters are just regular python strings or numbers.

### Composition
>Existing building blocks are reused as much as possible. For example, it is easy to create a pipeline estimator from an aribtrary sequence of transformers followed by an estimator.

### Sensible Defaults
>Scikit-Learn provides reasonable default values for most parameters, making it easy to quickly create a basline working system.

# Nested Brackets
When you see df[['col_name']] you're really seeing:

>col_names = ['col_name'] <br> df[col_names]

In consequence, the only thing that [[ does for you is that it makes the result a DataFrame, rather than a Series.

In [ ]:
housing_cat = housing[['ocean_proximity']]
housing_cat.head()

In [ ]:
# The problem with Scikit-Learn is, it heavily requires numerical data to work with.
# So we must convert the string to numbers, OrdinalEncoder in SKlearn helps with this
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
# To get the categories of the above dataset, use categories_
ordinal_encoder.categories_

# What is OneHotEncoder?
An issue with ML algorithms, they will assume that two nearby values are more similar than two distant values. But this isn't the case for ocean_proximity. As you can see above, cat 0 and cat 4 are for more related than 0 and 1. This image helps explain OneHotEncoder
<img src="img/one_hot_encoder_example.png" alt="Drawing" style="width: 500px;"/>

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot.toarray()

# Custom Transformers
Many times you will need to write your own for tasks such as custom cleanup operations or combining specific attributes. You need the transofrmer to work seamlessly with SciKit-Learn functionalities (like pipelines) and since Scikit-Learn relies on duck typing (not inheritence), all you need to do is create a class and implement three methods: fit() (returning self), transform(), and fit_transform().

> What is duck typing? <br> The idea of duck typing, is that you don't need a type in order to invoke an existing method on an object - if a method is defined on it, you can invoke it. <br> The name comes from the phrase "If it looks like a duck and quacks like a duck, it's a duck".

You can obtain fit_transform() by adding TransformerMixin as a base class. If you add BaseEstimator as a base class, you will get two extra methods (get_params() and set_params() that will be useful for automatic hyperparameter tuning.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [ ]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
housing_extra_attribs

# Feature Scaling
There are two common ways to get all attributes to have the same scale: min-max scaling and standardization

>min-max scaling (normalization) <br> min-max scaling (also called nomarlization) is the simplest: values are shifted and rescaled so that they end up ranging from 0 to 1. We do this by subtracting the min value and dividing by the max minus the min. Scikit-LEarn provides a transformer called MinMaxScaler for this. It has a feature_range hyperparameter that lets you change the range if, for some reason, you don't want 0 to 1.

>standardization <br> standardization first subtracts the mean value (so standardized values always have a zero mean), and then it divides by the standard deviation so that the resulting distribution has unit variance. Unlike min-max scaling, standardization does not bound values to a specific range, which may be a problem for some algorithms (e.g., neural networks often expect an input value ranging from 0 to 1). However, standardizing is much less affected by outliers. For example, lets say you input a median income of 100 (by mistake). Min-max scaling would crush all the other values from 0-15 down to 0.15. But standardization it would be much less affected. Scikit learn provides StandardScaler for standardization.

In [ ]:
# The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters.

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

housing_num_tr = num_pipeline.fit_transform(housing_numerical)

In [ ]:
# Pipelines
from sklearn.compose import ColumnTransformer

# Gets a list of numerical column names (i.e., we already defined all numerical columns above)
num_attribs = list(housing_numerical)
# Gets a list of category colunmn names
cat_attribs = ["ocean_proximity"]

# Handles all columns, applying the appropriate transformations to each column
# The constructor requires a list of tuples, where each tuple contains a name, a transformer, 
# and a list of names (or indices) of columns that the transformer should be applied to.
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),   #returns a dense matrix
    ('cat', OneHotEncoder(), cat_attribs) #returns a sparse matrix
])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
# Now that everything has been preprocessed and played with the syntax, we can train a model.
from sklearn.linear_model import LinearRegression

# A reminder, housing_prepared does not include median house price, that exists in housing_labels.
# So we fitting a model the solutions to the answers (housing_prepared=solutions, housing_labels=answer)
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("predictions: ", lin_reg.predict(some_data_prepared))
print("labels: ", list(some_labels))

In [ ]:
# MEasure the regression model's RMSE on the whole training set using scikit-learns mean_Squared_error() function
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

# _rmse = typical prediction error. As of running this, 68,628. This means the model can be off by that amount

# This shows the model underfitting the training data. This can mean that the features do not provide enough information
# to make good predictions, or that the model is not powerful enough

In [ ]:
# Here we will train a Devision tree regressor, which is a more powerful model, capable of finding complex nonlinear
# relationships in the data. 
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
# Again, fitting the model by giving the solution and the answers
tree_reg.fit(housing_prepared, housing_labels)


In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

# Cross-validation

We can seriously doubt that the model is 100% accurate. So we can use tran_test_split and break the data up into smaller training set and a validation set. Then we can train the model against the smaller training set and evaluate them against the validation se.

## HOWEVER!

Scikit-learn provides an alternative, K-fold cross-validation feature. This randomly splits the training set into 10 distinct subsets called folds. Then it trains and evaluated the decision tree model 10 times, picking a different fold for ecaluation every time and trining on the other 9 folds.

In [ ]:
# Here we will implement cross-validation tool
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())
    
display_scores(tree_rmse_scores)
# The Decision Tree is performing worse than the linear model!
# with a mean of 71270, +-2946

In [ ]:
# Lets compute the same type of score for the linear model
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
# Fitting the model, RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
# predicting and calculating the root mean square error
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)

In [ ]:
# Obtaining the score
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)
forest_rmse # Displays the prediction error, in this case, about $18000 off.

In [ ]:
# Instead of messing with hyperparameters manually, scikit-learn GridSearchSV can search for you.
# Just tell which hyperparameter you wish to experiment with.
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]
forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_
# Output: {'max_features': 8, 'n_estimators': 30}
grid_search.best_estimator_
# Output: RandomForestRegressor(max_features=8, n_estimators=30)

In [ ]:
# Shows the best features for accurate predictions
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
# Displays the importance scores next to their corresponding attribute names
extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
# Evaluate the final mode - test set
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)
# mean squared error, accepts answer, than solutions
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
# can computer 95% confidence interval for the generalization error using the following
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions - y_test) **2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                        loc=squared_errors.mean(),
                        scale=stats.sem(squared_errors)))